# test_02_remote

In [ ]:
#|default_exp test_02_remote
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import tests as this_module

Environment variable DISABLE_NBLITE_EXPORT is set to True, skipping export.


In [ ]:
#|top_export
import subprocess
from pathlib import Path
import shutil
import toml
import pytest
import asyncio

from repoyard import const
from repoyard.cmds import *
from repoyard._models import get_repoyard_meta
from repoyard.config import get_config

from tests.utils import *

from dotenv import load_dotenv

In [ ]:
#|top_export
def test_02_remote():
    asyncio.run(_test_02_remote())

In [ ]:
#|set_func_signature
async def _test_02_remote(): ...

Load config from env var. If it doesn't exist then skip test.

In [ ]:
#|export
from dotenv import load_dotenv
load_dotenv()
import os

if 'TEST_CONF_PATH' not in os.environ or 'TEST_STORAGE_LOCATION_NAME' not in os.environ or 'TEST_STORAGE_LOCATION_STORE_PATH' not in os.environ:
    pytest.skip("Environment variable TEST_CONF_PATH or TEST_STORAGE_LOCATION_NAME or TEST_STORAGE_LOCATION_STORE_PATH not set.")
else:
    config_path = Path(os.environ['TEST_CONF_PATH']).expanduser().resolve()
    config = get_config(config_path)
    sl_name = os.environ['TEST_STORAGE_LOCATION_NAME']
    sl_store_path = os.environ['TEST_STORAGE_LOCATION_STORE_PATH']

Ensure `repoyard` is installed

In [ ]:
#|export
from tests.utils import run_cmd, run_cmd_in_background, CmdFailed

try:
    run_cmd("repoyard")
except CmdFailed:
    pytest.skip("repoyard not installed")

Create repo and sync it

In [ ]:
#|export
repo_full_name1 = run_cmd(f"repoyard new -r test-repo-1 -g repoyard-unit-tests -s {sl_name}").strip()
run_cmd(f"repoyard sync -r {repo_full_name1}", capture_output=True);

Create two other repos and see they can both sync at the same time (i.e. test if simultaneous rclone commands can be run)

In [ ]:
#|export
repo_full_name2 = run_cmd(f"repoyard new -r test-repo-2 -g repoyard-unit-tests -s {sl_name}").strip()
repo_full_name3 = run_cmd(f"repoyard new -r test-repo-3 -g repoyard-unit-tests -s {sl_name}").strip()

p1 = run_cmd_in_background(f"repoyard sync -r {repo_full_name2}", print_output=False)
p2 = run_cmd_in_background(f"repoyard sync -r {repo_full_name3}", print_output=False)

p1.wait()
p2.wait()

0

Verify that the repos are there on remote

In [ ]:
#|export
from repoyard._models import get_repoyard_meta
repoyard_meta = get_repoyard_meta(config, force_create=True)
repo_meta1 = repoyard_meta.by_full_name[repo_full_name1]
repo_meta2 = repoyard_meta.by_full_name[repo_full_name2]
repo_meta3 = repoyard_meta.by_full_name[repo_full_name3]

from repoyard._utils import rclone_lsjson
for repo_meta in [repo_meta1, repo_meta2, repo_meta3]:
    assert rclone_lsjson(
        config.rclone_config_path,
        source=sl_name,
        source_path=repo_meta.get_remote_path(config),
    ) is not None

/var/folders/gc/wx3wyw4538vdtn_n2dyjn80m0000gn/T/ipykernel_29349/1244057810.py:10: RuntimeWarning: coroutine 'rclone_lsjson' was never awaited
  assert rclone_lsjson(


Exclude repos

In [ ]:
#|export
for repo_meta in [repo_meta1, repo_meta2, repo_meta3]:
    run_cmd(f"repoyard exclude -r {repo_meta.full_name}")

In [ ]:
#|export
async def _task(repo_meta):
    assert await rclone_lsjson(
        config.rclone_config_path,
        source="",
        source_path=repo_meta.get_local_repodata_path(config),
    ) is None

await asyncio.gather(*[_task(repo_meta) for repo_meta in [repo_meta1, repo_meta2, repo_meta3]]);

Re-include repos

In [ ]:
#|export
for repo_meta in [repo_meta1, repo_meta2, repo_meta3]:
    run_cmd(f"repoyard include -r {repo_meta.full_name}")

In [ ]:
#|export
async def _task(repo_meta):
    assert await rclone_lsjson(
        config.rclone_config_path,
        source="",
        source_path=repo_meta.get_local_repodata_path(config),
    ) is not None

await asyncio.gather(*[_task(repo_meta) for repo_meta in [repo_meta1, repo_meta2, repo_meta3]]);

Delete repos

In [ ]:
#|export
for repo_meta in [repo_meta1, repo_meta2, repo_meta3]:
    run_cmd(f"repoyard delete -r {repo_meta.full_name}")

In [ ]:
#|export
async def _task(repo_meta):
    assert await rclone_lsjson(
        config.rclone_config_path,
        source=sl_name,
        source_path=repo_meta.get_remote_path(config),
    ) is None

await asyncio.gather(*[_task(repo_meta) for repo_meta in [repo_meta1, repo_meta2, repo_meta3]]);